# EWC

$$
\mathcal{L} = 
$$

In [1]:
%load_ext autoreload
%autoreload 2

# Libraries

In [2]:
import pickle
from tqdm import tqdm
from itertools import product

In [3]:
from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.models import SimpleMLP
from avalanche.training import EWC
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import InteractiveLogger
from avalanche.evaluation.metrics import (
    accuracy_metrics,
    bwt_metrics,
    forgetting_metrics,
)

from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch import cuda, device

/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Custom Libraries

In [4]:
import sys

sys.path.append("../base_code/")

from base_code.constants import DATASETS_PATH, SAVED_METRICS_PATH
from base_code.seed import set_seed

# Constants

In [5]:
TORCH_DEVICE = device("cuda" if cuda.is_available() else "cpu")
N_EXPERIENCES = 10
EXPERIMENT_SEED = 1234
METRICS_SUBFOLDER = f"retraining_saturation"
METRICS_FILENAME = f"{METRICS_SUBFOLDER}/ewc.pkl"
set_seed(EXPERIMENT_SEED)

# Dataset and definitions

## Dataset loading

We load state-of-the-art dataset Modified NIST

In [6]:
scenario = PermutedMNIST(N_EXPERIENCES, seed=EXPERIMENT_SEED, dataset_root=DATASETS_PATH)

## Scenario creation with train test streaming

In this point, we define our scenario considering a training where in every experience of it, a new class is presented. This is, first we train with a class $a$, the following experience we train with class $b$ ($a \neq b$)

In [7]:
train_stream = scenario.train_stream
test_stream = scenario.test_stream

In [8]:
best_hyperparameters = {
    'ewc_lambda': 50.0
}

## Evaluation metrics definition

In [9]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True),
    forgetting_metrics(experience=True, stream=True),
    bwt_metrics(experience=True, stream=True),
    
    loggers=[InteractiveLogger()]
)

## Model, Optimizer, Loss function and Strategy definition

* `model`: Multi Layer Perceptron
* `Optimizer`: Adam
* `Loss function`: Cross Entropy
* `Strategy`: Elastic Weight Consolidation

In [10]:
model = SimpleMLP(num_classes=scenario.n_classes, input_size=28 * 28, hidden_layers=2, hidden_size=100).to(TORCH_DEVICE)
optimizer = SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=0.0005)
criterion = CrossEntropyLoss().to(TORCH_DEVICE)
strategy = EWC(
    model,
    optimizer,
    criterion,
    train_epochs=5,
    train_mb_size=128,
    evaluator=eval_plugin,
    eval_mb_size=128,
    device=TORCH_DEVICE,
    **best_hyperparameters
)

# Training and evaluation

Revisar porque el entrenamiento se está comportando de forma rara

In [11]:
training_exps = product(range(5), train_stream)

for (_, experience) in tqdm(training_exps):
    strategy.train(experience)
    strategy.eval(test_stream)

0it [00:00, ?it/s]

-- >> Start of training phase << --
100%|██████████| 469/469 [00:05<00:00, 90.80it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:04<00:00, 94.71it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:05<00:00, 87.80it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:05<00:00, 89.40it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:05<00:00, 88.10it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 102.75it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9109
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 100.96it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.1026
-- Starting eval on experience 2 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 98.32it/s] 
> Eval

1it [00:38, 38.34s/it]


> Eval on experience 9 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp009 = 0.1208
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = 0.0000
	StreamForgetting/eval_phase/test_stream = 0.0000
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.1941
-- >> Start of training phase << --
100%|██████████| 469/469 [00:05<00:00, 84.01it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:05<00:00, 83.70it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:06<00:00, 77.85it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:05<00:00, 82.09it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:05<00:00, 82.81it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 99.60it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.0133
	ExperienceForgetting/eval_phase/test_stream/T

2it [01:19, 39.83s/it]

...

-- >> Start of training phase << --
100%|██████████| 469/469 [00:15<00:00, 31.15it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:15<00:00, 31.10it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:15<00:00, 31.20it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:15<00:00, 30.79it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:15<00:00, 30.50it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 123.52it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.0276
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp000 = 0.0276
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.8902
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 124.02it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/

49it [51:34, 85.84s/it]


> Eval on experience 9 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp009 = -0.1986
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp009 = 0.1986
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp009 = 0.7105
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.0939
	StreamForgetting/eval_phase/test_stream = 0.0939
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.8324
-- >> Start of training phase << --
100%|██████████| 469/469 [00:15<00:00, 29.76it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:15<00:00, 29.53it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:15<00:00, 29.61it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:15<00:00, 29.87it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:15<00:00, 29.51it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 121.17it/s]
> Eval on experience 0 (Task 0)

50it [53:03, 63.67s/it]


> Eval on experience 9 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp009 = 0.9066
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.0896
	StreamForgetting/eval_phase/test_stream = 0.0896
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.8360


# Get metrics

In [12]:
accuracies: dict[int, list[float]] = dict()
forgettings: dict[int, list[float]] = dict()
bwt: dict[int, list[float]] = dict()

for i in range(N_EXPERIENCES):
    filled_i = str(i).zfill(3)
    accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    forgettings[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceForgetting/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    bwt[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceBWT/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
        

accuracies["Overall"] = eval_plugin.get_all_metrics()[
    "Top1_Acc_Stream/eval_phase/test_stream/Task000"
][1]

# Store metrics

In [14]:
pickle.dump({
    "accuracies": accuracies,
    "forgettings": forgettings,
    "bwt": bwt,
}, open(SAVED_METRICS_PATH / METRICS_FILENAME, "wb"))